In [ ]:
import nbimporter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from A_Label_AKI_Onsets import get_data_path, concat_dfs_to_one
import warnings
import contextlib
import os
import sys
import string
import statsmodels.formula.api as smf
from scipy.stats import norm
warnings.simplefilter(action="...", category=FutureWarning)
tqdm.pandas()
pd.set_option("...", None)

%store -r ct_names
%store -r raw_path
%store -r pat_id_cols
%store -r figure_dpi

In [ ]:
raw_path

In [ ]:
pat_id_cols

# Read All Patients

In [ ]:
all_patients = pd.read_csv("...")

In [ ]:
all_patients["..."] = pd.to_datetime(all_patients["..."])
all_patients["..."] = pd.to_datetime(all_patients["..."])
all_patients["..."] = pd.to_datetime(all_patients["..."])

In [ ]:
def find_highest_CKD_stage(row):
    if row["..."] == 1:
        return 5
    elif row["..."] == 1:
        return 4
    elif row["..."] == 1:
        return 3
    elif row["..."] == 1:
        return 2
    elif row["..."] == 1:
        return 1
    else:
        return 0
    
all_patients["..."] = all_patients.apply(find_highest_CKD_stage, axis = 1)

In [ ]:
# format data
all_patients[pat_id_cols] = all_patients[pat_id_cols].astype(str)

time_cols = ["...", "...", "..."]
for col in time_cols:
    all_patients[col] = pd.to_datetime(all_patients[col], format = "...")

In [ ]:
# is black will be a covariate to adjust
all_patients["..."] = all_patients["..."] == "..."
all_patients["..."] = all_patients["..."].astype(int)

In [ ]:
all_patients["..."] = all_patients["..."] + pd.Timedelta(days=365)
all_patients["..."] = all_patients["..."] + pd.Timedelta(days=30)

In [ ]:
all_patients

In [ ]:
all_patients

# Cluster Name Mapping

In case we enter wrong cluster

In [ ]:
cluster_mapping = {
    "...": 3,
    "...": 4,
    "...": 5,
    "...": 0,
    "...": 1,
    "...": 2,
}

In [ ]:
reversed_cluster_mapping = dict()
for name, val in cluster_mapping.items():
    reversed_cluster_mapping[val] = name

# Read Procedures

In [ ]:
from B_Data_Preprocessing import read_procedures, get_enc_by_PX_code

In [ ]:
Dia_RRT_codes = {
    "...": ["...","...","...", "...", 
          "...","...","...","...","...","...", "..."],
    
    "...": ["...","...","...","...","...", "...",
           "...","...","...","...","...","...","...","...","...","...","...", "..."],
    
    "...": [str(cpt) for cpt in range(90935, 91000)] + \
    ["...","...","...","...","...","...","...","...","...","...","...","..."]
}

In [ ]:
PX_dict = read_procedures(ct_names, raw_path)

In [ ]:
# return encounters related to the input code dict
def get_enc_by_future_PX_code(PX_dict, pat_df, code_dict, code_types, pat_id_cols):
    processed_PX_dict = dict()
    
    for ct_name, PX_df in tqdm(PX_dict.items()):
        # format type
        PX_df[["...", "...", "..."]] = PX_df[["...", "...", "..."]].astype(str)
        PX_df["..."] = PX_df["..."].replace("...", "...")
        PX_df["..."] = PX_df["..."].replace("...", "...")
        PX_df["..."] = PX_df["..."].replace("...", "...")
        
        # we only care about code-related PX, after that we format time
        PX_in_codes = []
        for code_type in code_types:
            PX_df_temp = PX_df[(PX_df.PX_TYPE == code_type) & (PX_df.PX.isin(code_dict[code_type]))]
            PX_in_codes.append(PX_df_temp)
            
        PX_df = pd.concat(PX_in_codes, axis = 0)
        
        pat_ct_df = pat_df[pat_df.CENTER_NAME == ct_name]
        pat_ct_df = pat_ct_df.merge(PX_df, on = "...", how = "...")
        
        #drop rows do not involed in the codes
        pat_ct_df.dropna(subset=["..."], inplace = True)
        
        
        # format time cols so that we can filter "..." dx later
        pat_ct_df["..."] = pd.to_datetime(pat_ct_df["..."], format = "...")
        pat_ct_df["..."] = pat_ct_df["..."].dt.strftime("...")
        pat_ct_df["..."] = pd.to_datetime(pat_ct_df["..."], format = "...")


        # require that it is after discharge
        pat_ct_df = pat_ct_df[pat_ct_df.PX_DATE > pat_ct_df.DISCHARGE_DATE]
        pat_ct_df = pat_ct_df[pat_id_cols + ["...", "...", "..."]]
        
        # keep useful info
        processed_PX_dict[ct_name] = pat_ct_df
        
    return processed_PX_dict

In [ ]:
# for each encounter's future PX, we only take the earliest one
def merge_one_PX_df(processed_PX_dict, pat_id_cols):
    processed_PX_df = []
    
    for ct_name, PX_df in processed_PX_dict.items():
        PX_df["..."] = (PX_df["..."] - PX_df["..."]).dt.days
        grouped = PX_df.groupby(pat_id_cols)
        idx = grouped["..."].idxmin()
        result_df = PX_df.loc[idx, pat_id_cols + ["...", "..."]]
        processed_PX_df.append(result_df)
    return pd.concat(processed_PX_df, axis = 0)

In [ ]:
future_dialysis_RRT = get_enc_by_future_PX_code(PX_dict, all_patients, 
                          Dia_RRT_codes, ["...", "...", "..."], 
                          pat_id_cols)

In [ ]:
DIA_RRT_df = merge_one_PX_df(future_dialysis_RRT, pat_id_cols)

In [ ]:
assert(len(DIA_RRT_df.ONSETS_ENCOUNTERID.unique()) == len(DIA_RRT_df))

# Merge Procedures and Process Data

In [ ]:
# format data
all_patients = all_patients.merge(DIA_RRT_df, on = pat_id_cols, how = "...")

In [ ]:
all_patients["..."] = (all_patients["..."] < all_patients["..."]).astype(int)

# Read Death Info

In [ ]:
def read_and_format_death(ct_names, raw_path):
    death_dict = read_Death(ct_names, raw_path)
    processed_death_dict = process_death(death_dict, ct_names)
    death_df = concat_dfs_to_one(processed_death_dict)
    return death_df

In [ ]:
def read_Death(ct_names, raw_path):
    
    death_dict = dict()
    use_cols = ["...", "..."]
    
    for ct_name in ct_names:
        
        data_path = get_data_path(ct_name, raw_path)
        
        if  (ct_name == "...") or (ct_name == "...") or (ct_name == "..."):
            death_df = pd.read_csv(data_path + "...", delimiter = "...", usecols = use_cols)
        elif (ct_name == "..."):
            death_df = pd.read_csv(data_path + "...", delimiter = "...", 
                                   usecols = ["...", "..."])
            
        elif (ct_name == "..."):
            death_df = pd.read_csv(data_path + "...", delimiter = "...",  
                                   usecols = ["...", "..."])
        
        elif (ct_name == "..."):
            death_df = pd.read_csv(data_path + "...", delimiter = "...", usecols = use_cols)

        elif (ct_name == "..."):
            death_df = pd.read_csv(data_path + "...", delimiter = "...", usecols = use_cols)

        elif (ct_name == "..."):
            death_df = pd.read_csv(data_path + "...", delimiter = "...", usecols = use_cols)
        
        death_df.columns = use_cols
        death_dict[ct_name] = death_df
    
    return death_dict

In [ ]:
def process_death(death_dict, ct_names):
    
    processed_death_dict = dict()
    
    for ct_name in ct_names:
        this_ct_death = death_dict[ct_name]
        if ct_name == "...":
            this_ct_death["..."] = pd.to_numeric(this_ct_death["..."], errors="...").astype("...")
        this_ct_death["..."] = this_ct_death["..."].astype(str)
        this_ct_death["..."] = pd.to_datetime(this_ct_death["..."], 
                                                     format = "...")
        if ct_name == "...":
            this_ct_death["..."] = this_ct_death["..."].dt.date
            this_ct_death["..."] = pd.to_datetime(this_ct_death["..."])
            
        this_ct_death["..."] = ct_name
        processed_death_dict[ct_name] = this_ct_death
    
    return processed_death_dict        

In [ ]:
death_df = read_and_format_death(ct_names, raw_path)

In [ ]:
death_df.drop_duplicates(subset = ["...", "..."], inplace = True)

# Read BMI Info and Merge

In [ ]:
def read_and_format_BMI(ct_names, raw_path):
    vital_dict = read_Vital(ct_names, raw_path)
    processed_vital_dict = process_Vital(vital_dict, ct_names)
    BMI_df = concat_dfs_to_one(processed_vital_dict)
    return BMI_df

In [ ]:
def read_Vital(ct_names, raw_path):
    vital_dict = dict()
    for ct_name in ct_names:
        
        data_path = get_data_path(ct_name, raw_path)
        
        if (ct_name == "...") or (ct_name == "..."):
            vital_df = pd.read_csv(data_path + "...", delimiter = "...")
        elif (ct_name == "...") or (ct_name == "..."):
            vital_df = pd.read_csv(data_path + "...", delimiter = "...")
        elif (ct_name == "...") or (ct_name == "..."):
            vital_df = pd.read_csv(data_path + "...", delimiter = "...")
            vital_df.columns = vital_df.columns.str.upper()
        elif (ct_name == "..."):
            vital_df = pd.read_csv(data_path + "...", delimiter = "...")
        elif (ct_name == "..."):
            vital_df = pd.read_csv(data_path + "...", delimiter = "...")
            
        vital_dict[ct_name] = vital_df
    return vital_dict

In [ ]:
def calculate_BMI(row):
    WT, HT = row["..."], row["..."]
    BMI = (WT / (HT ** 2)) * 703
    return BMI

In [ ]:
def process_Vital(vital_dict, ct_names):
    processed_vital_dict = dict()
    for ct_name in ct_names:
        this_vital = vital_dict[ct_name]
        this_vital[["...", "..."]] = this_vital[["...", "..."]].astype(str)
        
        if (ct_name == "...") or (ct_name == "..."):
            BMI = this_vital.groupby(["...", "..."], as_index=False)["..."].mean().reset_index(drop = True)
        if (ct_name == "..."):
            this_vital.columns = this_vital.columns.str.upper()
            BMI = this_vital.groupby(["...", "..."], as_index=False)["..."].mean().reset_index(drop = True)
        elif (ct_name == "...") or (ct_name == "..."):
            BMI = this_vital[this_vital.OBSCLIN_CODE == "..."].groupby(["...", "..."], as_index=False)["..."].mean()
        elif (ct_name == "..."):
            this_vital["..."] = this_vital["..."].str[1:-1]
            this_vital["..."] = this_vital["..."].str[1:-1]
            BMI = this_vital.groupby(["...", "..."], as_index=False)["..."].mean().reset_index(drop = True)
        elif (ct_name == "..."):
            BMI = this_vital[this_vital.OBSCLIN_CODE == \
                            "..."].groupby(["...", "..."], as_index=False)["..."].mean().reset_index(drop = True)
        elif (ct_name == "..."):
            BMI_info = this_vital.groupby(["...", "..."], as_index=False)[["...", "..."]].mean().reset_index(drop = True)
            BMI_info["..."] = BMI_info.apply(calculate_BMI, axis = 1)
            BMI = BMI_info.loc[:, ["...", "...", "..."]]
            
        BMI.columns = ["...", "...", "..."]
        BMI["..."] = ct_name
        processed_vital_dict[ct_name] = BMI
        
    return processed_vital_dict   

In [ ]:
vital_df = read_and_format_BMI(ct_names, raw_path)

In [ ]:
all_patients = all_patients.merge(vital_df, on = pat_id_cols, how = "...")

# Merge Death and Process Data

In [ ]:
# format data
all_patients = all_patients.merge(death_df, on = ["...", "..."], how = "...")

In [ ]:
# sanity check
for ct_name in ct_names:
    assert(np.sum(all_patients[all_patients.CENTER_NAME == ct_name]["..."].notna()) > 0)

In [ ]:
all_patients["..."] = (all_patients["..."] < all_patients["..."]).astype(int)
all_patients["..."] = (all_patients["..."] < all_patients["..."]).astype(int)
all_patients["..."] = (all_patients["..."] - all_patients["..."]).dt.days

# Label SCr Recovery

In [ ]:
from A_Label_AKI_Onsets import read_and_format_SCR

In [ ]:
SCR_df = read_and_format_SCR(ct_names, raw_path)

In [ ]:
# multiple measurements on the same day averaged
SCR_df = SCR_df.sort_values(by=["...", "...", "..."])
SCR_df = SCR_df.groupby(["...", "...", "..."]).RESULT_NUM.mean().reset_index()

In [ ]:
AKI_patients = all_patients.loc[all_patients.NONAKI_SINCE_ADMIT == False, :]

In [ ]:
AKI_patients_id = AKI_patients[pat_id_cols + ["...", "...", "..."]].copy(deep = True)
AKI_patients_SCR = AKI_patients_id.merge(SCR_df, on = ["...", "..."], how = "...")

In [ ]:
# get SCr measurements after onset and before discharge
AKI_patients_SCR = AKI_patients_SCR.loc[(AKI_patients_SCR.SPECIMEN_DATE > AKI_patients_SCR.AKI1_ONSET) & \
                                    (AKI_patients_SCR.SPECIMEN_DATE <= AKI_patients_SCR.DISCHARGE_DATE), :]

In [ ]:
# screen those SCr values below 1.5 * SCr baseline (do not satisfy AKI-1)
recovery_SCR = AKI_patients_SCR.loc[AKI_patients_SCR.RESULT_NUM <= 1.5 * AKI_patients_SCR.BASELINE_SCR, :]

In [ ]:
# only keep the earliest AKI recovery time
recovery_SCR = recovery_SCR.groupby(pat_id_cols).first().reset_index()

In [ ]:
recovery_SCR.loc[:, "..."] = (recovery_SCR.SPECIMEN_DATE - recovery_SCR.AKI1_ONSET).dt.days
recovery_SCR.loc[:, "..."] = recovery_SCR.loc[:, "..."]
recovery_SCR.loc[recovery_SCR.SEVEN_DAYS_SCR_RECOVERY_TIME > 7, "..."] = 7
recovery_SCR.loc[:, "..."] = 0
recovery_SCR.loc[recovery_SCR.SCR_RECOVERY_TIME <= 7, "..."] = 1

In [ ]:
AKI_patients = AKI_patients.merge(recovery_SCR[pat_id_cols + ["...", "..."]],
                                 on = pat_id_cols, how = "...")

In [ ]:
AKI_patients["..."] = AKI_patients["..."].fillna(0)
AKI_patients["..."] = AKI_patients["..."].fillna(7)

In [ ]:
all_patients = all_patients.merge(AKI_patients[pat_id_cols + ["...", "..."]],
                                 on = pat_id_cols, how = "...")

# Save the Table

In [ ]:
all_patients.to_csv("...", index = False)

# Compare AKI Subtypes by Hazard Ratio

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
all_patients = pd.read_csv("...")
all_patients["..."] = pd.to_datetime(all_patients["..."])
all_patients["..."] = pd.to_datetime(all_patients["..."])
all_patients["..."] = pd.to_datetime(all_patients["..."])
all_patients[pat_id_cols] = all_patients[pat_id_cols].astype(str)

In [ ]:
# save the age cy, before normalization, as we need this for table 1
age_distr = {
    "...": {},
    "...": {},
    "...": {},
}

for name in cluster_names:
    cluster_idx = cluster_mapping[name]
    this_cluster = all_patients[all_patients.CLUSTERS == cluster_idx]
    age_distr["..."][name] = len(this_cluster[(this_cluster.AGE >= 18) & \
                                                          (this_cluster.AGE < 30)])
    age_distr["..."][name] = len(this_cluster[(this_cluster.AGE >= 30) & \
                                                          (this_cluster.AGE < 60)])
    age_distr["..."][name] = len(this_cluster[this_cluster.AGE >= 60])

In [ ]:
# Columns to normalize
cols_to_normalize = ["...", "..."]

for col in cols_to_normalize:
    # Compute 1st and 99th percentiles to define robust min/max
    q_low = all_patients[col].quantile(0.01)
    q_high = all_patients[col].quantile(0.99)
    
    # Min-max normalization using robust range
    all_patients[col] = (all_patients[col] - q_low) / (q_high - q_low)
    
    # Clip to [0, 1] range
    all_patients[col] = all_patients[col].clip(0, 1)

In [ ]:
adjust_variables_1 = []
adjust_variables_2 = ["...", "..."]
adjust_variables_3 = adjust_variables_2 + ["...", "...", "...", "..."]
adjust_variables_4 = adjust_variables_3 + ["..."]

In [ ]:
all_AKI_patients = all_patients[all_patients.CLUSTERS.isin([cluster_mapping["..."], cluster_mapping["..."], cluster_mapping["..."]])]
all_AKI_patients = all_AKI_patients[adjust_variables_4+["...", "...", 
                                                        "...", "...", "..."]].copy(deep = True)

In [ ]:
def compare_risk_among_AKI(patients, ref_group, adjust_variables, event_col, non_AKI_compare = False):

    # Build formula and fit model
    formula = construct_formula(ref_group, adjust_variables, event_col)
    LR_model = smf.logit(formula=formula, data=patients).fit(maxiter=50)

    # Extract summary
    summary = LR_model.summary2().tables[1]
    summary["..."] = np.exp(summary["..."])
    summary["..."] = np.exp(summary["..."] - 1.96 * summary["..."])
    summary["..."] = np.exp(summary["..."] + 1.96 * summary["..."])
    
    if non_AKI_compare == False:

        # Extract coefficients for 4 vs 3 and 5 vs 3
        beta_4 = LR_model.params["..." % ref_group]
        beta_5 = LR_model.params["..." % ref_group]

        # Delta method for 5 vs 4
        contrast = beta_5 - beta_4
        cov = LR_model.cov_params()
        var_4 = cov.loc["..." % ref_group,
                        "..." % ref_group]
        var_5 = cov.loc["..." % ref_group,
                        "..." % ref_group]
        cov_45 = cov.loc["..." % ref_group,
                         "..." % ref_group]
        se_contrast = np.sqrt(var_4 + var_5 - 2 * cov_45)

        # Calculate OR, CI, p-value
        or_5vs4 = np.exp(contrast)
        ci_lower = np.exp(contrast - 1.96 * se_contrast)
        ci_upper = np.exp(contrast + 1.96 * se_contrast)
        p_val = 2 * (1 - norm.cdf(np.abs(contrast / se_contrast)))

        # Prepare output
        result = summary.loc[
            [f"...",
             f"..."],
            ["...", "...", "...", "..."]
        ].copy()
        result.index = [f"...", f"..."]
        # Add 5 vs 4 comparison
        result.loc["..."] = [or_5vs4, p_val, ci_lower, ci_upper]
    else:
        cluster_prefix = f"..."
        mask = summary.index.to_series().str.startswith(cluster_prefix)
        result = summary.loc[mask, ["...", "...", "...", "..."]].copy()
        
    return result

In [ ]:
def construct_formula(ref_group, adjust_variables, event_col):
    adjustment = "..."
    for var in adjust_variables:
        adjustment += "..."%(var)
    formula = event_col + "..."%(ref_group) + adjustment
    return formula

# Populate the Outcome Table

In [ ]:
import warnings

In [ ]:
outcome_df = pd.DataFrame(index = ["...", 
                                   "...",
                                   "...",
                                   "...", "...",
                                   "...", "...", 
                                   "...", "...",
                                   "...", "...",
                                   "...", "...",
                                   "...", "...",
                                    "...", "...",
                                   "...", "..."],
                          
                         columns = ["...", "...", "...",
                                    "...", "...", "..."])

In [ ]:
# we do not compare SCr recovry between AKI and non-AKI
outcome_df.loc[[ "...", "...",
                "..."],
              ["...", "...", "..."]] = "..."

In [ ]:
def populate_AKI_outcome(outcome_df, all_AKI_patients, target_list, adjust_var_list):
    for tgt in target_list:
        for i, adj_var in enumerate(adjust_var_list):
            print(i)
            
            if tgt == "..." and i == 3:
                continue
                
            else:
                result = compare_risk_among_AKI(all_AKI_patients, cluster_mapping["..."], adj_var, tgt)
                
                # locate row
                if tgt == "...":
                    index_name_start = "..."
                elif tgt == "...":
                    index_name_start = "..."
                elif tgt == "...":
                    index_name_start = "..."
                elif tgt == "...":
                    index_name_start = "..."
                elif tgt == "...":
                    index_name_start = "..."
                    
                
                if i == 0:
                    model_name = "..."
                else:
                    model_name = "..." + str(i)
                
                row_name = index_name_start + "..." + model_name
                
                outcome_df.loc[row_name, "..."] = format_OR_CI_pval(result, "...")
                outcome_df.loc[row_name, "..."] = format_OR_CI_pval(result, "...")
                outcome_df.loc[row_name, "..."] = format_OR_CI_pval(result, "...")
    return outcome_df 

def format_OR_CI_pval(table, index_name):
    row = table.loc[index_name]
    OR = row["..."]
    CI_lower = row["..."]
    CI_upper = row["..."]
    p_val = row["..."]

    # Format output string
    formatted = f"..."
    return formatted

In [ ]:
with contextlib.redirect_stdout(open(os.devnull, "...")), \
     contextlib.redirect_stderr(open(os.devnull, "...")), \
     warnings.catch_warnings():
    
    warnings.simplefilter("...")     # Suppress warnings like ConvergenceWarning
    np.seterr(all="...")             # Suppress numpy warnings like overflow
    target_list = ["...", "...", "...", "..."]
    adjust_var_list = [adjust_variables_1, adjust_variables_2, adjust_variables_3, adjust_variables_4]
    outcome_df = populate_AKI_outcome(outcome_df, all_AKI_patients, target_list, adjust_var_list)

# Compare AKI vs Non-AKI

In [ ]:
def populate_AKI_vs_non_AKI_outcome(outcome_df, all_patients, target_list, adjust_var_list):
    
    
    ref_groups = ["...", "...", "..."]
    
    for ref in ref_groups:
        sub_pop = all_patients[all_patients.CLUSTERS.isin([cluster_mapping[ref[:-1]], cluster_mapping[ref]])].copy(deep = True)
        
        for tgt in target_list:
            for i, adj_var in enumerate(adjust_var_list):
                result = compare_risk_among_AKI(sub_pop, cluster_mapping[ref], adj_var, tgt, True)

                # locate row
                if tgt == "...":
                    index_name_start = "..."
                elif tgt == "...":
                    index_name_start = "..."
                elif tgt == "...":
                    index_name_start = "..."
                elif tgt == "...":
                    index_name_start = "..."

                if i == 0:
                    model_name = "..."
                else:
                    model_name = "..." + str(i)

                row_name = index_name_start + "..." + model_name

                outcome_df.loc[row_name, "..."%(ref, ref[:-1])] = format_OR_CI_pval(result, result.index[0])

    return outcome_df 

In [ ]:
with contextlib.redirect_stdout(open(os.devnull, "...")), \
     contextlib.redirect_stderr(open(os.devnull, "...")), \
     warnings.catch_warnings():
    warnings.simplefilter("...")     # Suppress warnings like ConvergenceWarning
    np.seterr(all="...")             # Suppress numpy warnings like overflow
    target_list = ["...", "...", "..."]
    adjust_var_list = [adjust_variables_1, adjust_variables_2, adjust_variables_3, adjust_variables_4]
    outcome_df = populate_AKI_vs_non_AKI_outcome(outcome_df, all_patients, target_list, adjust_var_list)

In [ ]:
outcome_df

# Read Labs

We want to know the lab status 3 days before onset. For non-AKI patients, the window is 3-days before the last SCR measurements, thus, we also need SCr.

In [ ]:
def read_Labs(ct_names, raw_path):
    
    LAB_dict = dict()
    use_cols = ["...", "...", "...", "..."]
    
    for ct_name in tqdm(ct_names):
        
        data_path = get_data_path(ct_name, raw_path)
        
        if ct_name == "...":
            with open(data_path + "...", "...", encoding="...", errors="...") as file:
                LAB_df = pd.read_csv(data_path + "...", 
                                  delimiter="...", usecols=["...", "...", "...", 
                                                          "..."],
                                  encoding="...")

        elif ct_name == "...":
            with open(data_path + "...", "...", encoding="...", errors="...") as file:
                LAB_df = pd.read_csv(data_path + "...", 
                                  delimiter="...", usecols=["...", "...", "...", "..."],
                                  encoding="...")

        elif ct_name == "...":
            with open(data_path + "...", "...", encoding="...", errors="...") as file:
                LAB_df = pd.read_csv(data_path + "...", 
                                  delimiter = "...", usecols = use_cols,
                                  encoding = "...")

        elif ct_name == "...":
            with open(data_path + "...", "...", encoding="...", errors="...") as file:
                LAB_df = pd.read_csv(data_path + "...", 
                                  delimiter = "...", usecols = use_cols,
                                  encoding = "...")
            LAB_df["..."] = pd.to_datetime(LAB_df["..."], format="...")
            LAB_df["..."] = LAB_df["..."].dt.date

        elif ct_name == "...":
            with open(data_path + "...", "...", encoding="...", errors="...") as file:
                LAB_df = pd.read_csv(data_path + "...", 
                                  delimiter="...", usecols = use_cols,
                                  encoding="...")

        elif (ct_name == "...") or (ct_name == "..."):
            LAB_df = pd.read_csv(data_path + "...", delimiter = "...", 
                              usecols = use_cols)

        elif ct_name == "...":
            with open(data_path + "...", "...", encoding="...", errors="...") as file:
                LAB_df = pd.read_csv(file, delimiter="...", 
                                  usecols = use_cols)


        LAB_df.columns = ["...", "...", "...", "..."]       
        LAB_dict[ct_name] = LAB_df
    return LAB_dict

In [ ]:
def process_Labs(pat_df, LAB_dict):
    processed_Lab_dict = dict()
    
    for ct_name, Lab_df in tqdm(LAB_dict.items()):
        # format type
        Lab_df[["...", "..."]] = Lab_df[["...", "..."]].astype(str)
        
        pat_ct_df = pat_df[pat_df.CENTER_NAME == ct_name][["...", "..."]]
        pat_ct_df = pat_ct_df.merge(Lab_df, on = "...", how = "...")
        
        pat_ct_df["..."] = pd.to_datetime(pat_ct_df["..."], format = "...")
        
        # get the lab status 7 days before lab collection point
        pat_ct_df_filtered = pat_ct_df[(pat_ct_df.SPECIMEN_DATE >= (pat_ct_df.LAB_WINDOW_POINT - pd.Timedelta(days=7))) & \
                                       (pat_ct_df.SPECIMEN_DATE < pat_ct_df.LAB_WINDOW_POINT)]
        
        pat_ct_df_filtered = pat_ct_df_filtered.sort_values(by=["...", "...","..."])
        pat_ct_df_filtered = pat_ct_df_filtered.groupby(["...", "..."])["..."].last().reset_index()
        pivot_table = pat_ct_df_filtered.pivot(index="...", columns="...", values="...").reset_index()
        pivot_table["..."] = ct_name
        processed_Lab_dict[ct_name] = pivot_table
    
    return processed_Lab_dict

In [ ]:
# now we can merge Labs to patients
Lab_dict = read_Labs(ct_names, raw_path)

In [ ]:
# we still separate AKI and non-AKI patients and collect their lab collection point separately
# for AKI patients, it is just AKI-1 onset date
# for non-AKI patients, it is the last SCr measurement time
AKI_patients = all_patients.loc[all_patients.NONAKI_SINCE_ADMIT == False, :].copy(deep = True)
Non_AKI_patients = all_patients.loc[all_patients.NONAKI_SINCE_ADMIT == True, :].copy(deep = True)
AKI_patients.loc[:, "..."] = AKI_patients.loc[:, "..."]
AKI_patients["..."] = pd.to_datetime(AKI_patients["..."])

In [ ]:
Non_AKI_patients_SCR = Non_AKI_patients.merge(SCR_df, on = ["...", "..."], how = "...")
# since SCR_df is already sorted, we do not sort here
# get SCr measurements during hospitalization
Non_AKI_patients_SCR = Non_AKI_patients_SCR.loc[(Non_AKI_patients_SCR.SPECIMEN_DATE > Non_AKI_patients_SCR.ADMIT_DATE) & \
                                    (Non_AKI_patients_SCR.SPECIMEN_DATE < Non_AKI_patients_SCR.DISCHARGE_DATE), :]
Non_AKI_patients_SCR = Non_AKI_patients_SCR.groupby(pat_id_cols)["..."].last().reset_index()

In [ ]:
Non_AKI_patients_SCR.rename(columns = {"...": "..."}, inplace = True)
Non_AKI_patients = Non_AKI_patients.merge(Non_AKI_patients_SCR, on = pat_id_cols, how = "...")
Non_AKI_patients["..."] = Non_AKI_patients["..."].fillna(Non_AKI_patients["..."])
Non_AKI_patients["..."] = pd.to_datetime(Non_AKI_patients["..."])

In [ ]:
all_patients = pd.concat([AKI_patients, Non_AKI_patients], axis = 0)

In [ ]:
processed_Lab_dict = process_Labs(all_patients, Lab_dict)

In [ ]:
Lab_df = concat_dfs_to_one(processed_Lab_dict)

In [ ]:
original_lab_space = list(Lab_df.columns)
original_lab_space.remove("...")
original_lab_space.remove("...")

In [ ]:
all_patients = all_patients.merge(Lab_df, on = ["...", "..."], how = "...")

In [ ]:
# drop labs with nan rate > 0.3
nan_ratios = all_patients[original_lab_space].isna().mean()
columns_to_drop = nan_ratios[nan_ratios > 0.3].index
# also drop SCr
columns_to_drop = list(columns_to_drop) + ["..."]
all_patients = all_patients.drop(columns=columns_to_drop)

In [ ]:
filtered_lab_space = []
for lab in original_lab_space:
    if lab not in list(columns_to_drop):
        filtered_lab_space.append(lab)

In [ ]:
loinc_to_lab_name = {
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
}

In [ ]:
all_patients.rename(columns = loinc_to_lab_name, inplace = True)
# now labs have been transformed into names
filtered_lab_space = list(loinc_to_lab_name.values())

For lab illustration, we will show one X plot and one table showing percentage of abnormalty and a p-value table

1. X Plots

In [ ]:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

In [ ]:
def plot_Standarized_Mean_Variable(chosen_2_cluster_idx, pat_df, use_cols,
                                   title, color_key, test_organ_map, organ_color_map, ax):
    
    clustering = pat_df[(pat_df["..."] == chosen_2_cluster_idx[0]) | \
                               (pat_df["..."] == chosen_2_cluster_idx[1])]
    
    clustering_for_X_plot = clustering[use_cols]
    
    
    #normalize frame
    norm_clustering_for_X_plot = (clustering_for_X_plot - clustering_for_X_plot.mean()) / clustering_for_X_plot.std()
    norm_clustering_for_X_plot["..."] = clustering["..."]
    
    #create helping table
    stat_table_for_X_plot = \
    pd.DataFrame(0, index = clustering_for_X_plot.columns, columns = chosen_2_cluster_idx)
    for test in clustering_for_X_plot.columns:
        for c in chosen_2_cluster_idx:
            norm_this_cluster = norm_clustering_for_X_plot[norm_clustering_for_X_plot["..."] == c]
            stat_table_for_X_plot.loc[test, c] = np.mean(norm_this_cluster[test])
    
    #helping col to sort rows so that X will show up
    stat_table_for_X_plot["..."] = \
        stat_table_for_X_plot[chosen_2_cluster_idx[0]] - stat_table_for_X_plot[chosen_2_cluster_idx[1]]
    stat_table_for_X_plot.sort_values(by="...", inplace = True)

    
    index = list(stat_table_for_X_plot.index)
    
    lines = []
    for c in chosen_2_cluster_idx:
        line, = ax.plot(stat_table_for_X_plot.loc[:, c], index, marker = "...", color = color_key[c])
        lines.append(line)
        
    #add some body organ/system marker on the y axis
    ytick_locations = ax.get_yticks()
    ytick_labels = [label.get_text() for label in ax.get_yticklabels()]
    yticks_combined = list(zip(ytick_locations, ytick_labels))

    for i, (ytick_loc, ytick_label) in enumerate(yticks_combined):
        marker = ax.scatter(-0.65, ytick_loc, marker="...", 
                                color=organ_color_map[test_organ_map[ytick_label]], zorder=5, s=50) 
        
    ax.axvline(x=0, color="...", lw = 0.4)
    ax.grid(True, lw = 0.15)
    ax.set_title(title)

In [ ]:
organ_color_map = {
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
}

test_organ_map = {
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
}

In [ ]:
# beacuse we switch the label, we need a new color key
new_color_key = {
    0: "...",
    1: "...",
    2: "...",
    3: "...",
    4: "...",
    5: "...",
    6: "...",
    7: "...",
    8: "..."
}

In [ ]:
# Use MICE to impute missing values
imputer = IterativeImputer(missing_values=np.nan, 
                           max_iter=10000, random_state=42)
# should only fit on train data in case for data leakage
all_patients.loc[:, filtered_lab_space] = \
imputer.fit_transform(all_patients.loc[:, filtered_lab_space])
all_patients.loc[:, filtered_lab_space] = all_patients.loc[:, filtered_lab_space].astype(np.float64)

In [ ]:
# make a legend
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

custom_lines = [
    Line2D([0], [0], color="...", lw=2, label="..."),
    Line2D([0], [0], color="...", lw=2, label="..."),
    Line2D([0], [0], color="...", lw=2, label="..."),
    Line2D([0], [0], color="...", lw=2, label="..."),
    Line2D([0], [0], color="...", lw=2, label="..."),
    Line2D([0], [0], color="...", lw=2, label="..."),
    Line2D([0], [0], color=organ_color_map["..."], lw=0, marker="...", markersize=10, label="..."),
    Line2D([0], [0], color=organ_color_map["..."], lw=0, marker="...", markersize=10, label="..."),
    Line2D([0], [0], color=organ_color_map["..."], lw=0, marker="...", markersize=10, label="...")
]

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("...")
    
    fig, axs = plt.subplots(2, 3, figsize=(12, 10))
    
    plot_Standarized_Mean_Variable([3, 4], all_patients, filtered_lab_space, "...", 
                                   new_color_key, test_organ_map, organ_color_map, axs[0, 0])

    plot_Standarized_Mean_Variable([4, 5], all_patients, filtered_lab_space, "...", 
                                   new_color_key, test_organ_map, organ_color_map, axs[0, 1])

    plot_Standarized_Mean_Variable([3, 5], all_patients, filtered_lab_space, "...", 
                                   new_color_key, test_organ_map, organ_color_map, axs[0, 2])

    plot_Standarized_Mean_Variable([0, 3], all_patients, filtered_lab_space, "...", 
                                   new_color_key, test_organ_map, organ_color_map, axs[1, 0])

    plot_Standarized_Mean_Variable([1, 4], all_patients, filtered_lab_space, "...", 
                                   new_color_key, test_organ_map, organ_color_map, axs[1, 1])

    plot_Standarized_Mean_Variable([2, 5], all_patients, filtered_lab_space, "...", 
                                   new_color_key, test_organ_map, organ_color_map, axs[1, 2])
   
    # Adding sequential labels (A, B, C, D) to each subplot
    labels = list(string.ascii_lowercase)
    positions = [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]

    for label, pos in zip(labels, positions):
        axs[pos].text(-0.1, 1.1, label, transform=axs[pos].transAxes, 
                      fontsize=16, fontweight="...", va="...", ha="...")

    plt.tight_layout()
    plt.legend(handles=custom_lines, loc="...", bbox_to_anchor=(0.864, 0.85), bbox_transform=plt.gcf().transFigure)
    plt.savefig("...", format="...", dpi = figure_dpi)
    plt.show()

2. Table and p-values 

In [ ]:
lab_normal_range = dict()
lab_normal_range["..."] = [8.5, 10.5]
lab_normal_range["..."] = [96, 106]
lab_normal_range["..."] = [0, 125]
lab_normal_range["..."] = [3.5, 5.5]
lab_normal_range["..."] = [135, 145]
lab_normal_range["..."] = [6, 24]
lab_normal_range["..."] = [38, 51]
lab_normal_range["..."] = [12.1, 18]
lab_normal_range["..."] = [0, 10]
lab_normal_range["..."] = [150, 450]
lab_normal_range["..."] = [27, 31]
lab_normal_range["..."] = [32, 36]
lab_normal_range["..."] = [80, 100]
lab_normal_range["..."] = [12, 15.3]
lab_normal_range["..."] = [4.1, 5.4]
assert(len(lab_normal_range) == len(filtered_lab_space))

In [ ]:
cluster_names = list(cluster_mapping.keys())

In [ ]:
lab_abnom_df = pd.DataFrame(0, index = ["..."] + filtered_lab_space, 
                          columns = ["...", *cluster_names[:len(cluster_names)//2], 
                                    "...", *cluster_names[len(cluster_names)//2:]])

In [ ]:
for lab in filtered_lab_space:
    normal_range_lower = lab_normal_range[lab][0]
    normal_range_upper = lab_normal_range[lab][1]
    
    for cluster_name in cluster_names:
        cluster_idx = cluster_mapping[cluster_name]
        this_cluster = all_patients.loc[all_patients.CLUSTERS == cluster_idx, :]
        this_cluster_lab_abnorm_n = ((this_cluster[lab] > normal_range_upper) | \
                                     (this_cluster[lab] < normal_range_lower)).sum()
        lab_abnom_df.loc[lab, cluster_name] = this_cluster_lab_abnorm_n

In [ ]:
# fill in total patient number
for cluster_name in cluster_names:
    cluster_idx = cluster_mapping[cluster_name]
    lab_abnom_df.loc["...", cluster_name] = \
    len(all_patients[all_patients.CLUSTERS == cluster_idx])

In [ ]:
lab_abnom_df.loc[:, "..."] = lab_abnom_df.loc[:, cluster_names[:len(cluster_names)//2]].sum(axis = 1)
lab_abnom_df.loc[:, "..."] = lab_abnom_df.loc[:, cluster_names[len(cluster_names)//2:]].sum(axis = 1)

In [ ]:
def add_percentage(col):
    total = col.iloc[0]  # first row (cluster total) is the total count
    return col.map(lambda entry: f"..." if total != 0 else "...")

In [ ]:
lab_abnom_df_final = lab_abnom_df.apply(add_percentage, axis = 0)

In [ ]:
lab_abnom_df_final

Calculate P values

In [ ]:
from scipy.stats import chi2_contingency

In [ ]:
def calculate_P_values(num_df, total_n_row_name = "..."):
    col_names, cluster_pairs = get_P_valuse_df_col_names(num_df)
    index_names = [idx for idx in num_df.index if idx != total_n_row_name]
    P_values_df = pd.DataFrame(0, index = index_names, columns = col_names)
    
    for i, pair in enumerate(cluster_pairs):
        pvales = []
        col_name = col_names[i]
        for catg in P_values_df.index:
            clusters_for_test = []
            clusters_for_test.append(num_df.loc[catg, pair[0]])
            clusters_for_test.append(num_df.loc[catg, pair[1]])
            cluster_total = num_df.loc[total_n_row_name, [pair[0], pair[1]]].values
            p = contingency_table_and_p_value(clusters_for_test, cluster_total)
            pvales.append(p)
        
        pvales_cats = ["...".format(pvalue) for pvalue in pvales]  
        P_values_df.loc[:, col_name] = pvales_cats

    return P_values_df

In [ ]:
def get_P_valuse_df_col_names(num_df, calculate_total = True):
    NONAKI_cols = [col for col in num_df.columns if (col.endswith("...")) and (col.startswith("..."))]
    AKI_cols = [col for col in num_df.columns if (col not in NONAKI_cols) and (col.startswith("..."))]
    
    cluster_pairs = []
    
    finished_cols = []
    for col1 in AKI_cols:
        for col2 in AKI_cols:
            if (col1 == col2) or (col2 in finished_cols):
                continue
            cluster_pairs.append((col1,col2))
        finished_cols.append(col1)
    
    assert(len(AKI_cols) == len(NONAKI_cols))
    for i in range(len(AKI_cols)):
        cluster_pairs.append((AKI_cols[i], NONAKI_cols[i]))
    
    if calculate_total:
        cluster_pairs.insert(0, ("...", "..."))
    
    p_val_cols_names = [cluster_pair[0] + "..." + cluster_pair[1] for cluster_pair in cluster_pairs]
    
    return p_val_cols_names, cluster_pairs

In [ ]:
def get_contingency_table(clusters_for_test, cluster_total):
    contingency_first_row = np.array(clusters_for_test)
    contingency_second_row = cluster_total - contingency_first_row
    contingency_table = np.vstack((contingency_first_row, contingency_second_row))
    return contingency_table

In [ ]:
def contingency_table_and_p_value(clusters_for_test, cluster_total):
    contingency_table = get_contingency_table(clusters_for_test, cluster_total)
            
    #when any input value for test is 0, p is 0, else, use chi2
    if np.any(np.array(clusters_for_test) == 0):
        p = 0
    else:
        _, p, _, _ = chi2_contingency(contingency_table)
    return p

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("...")
    lab_abnom_p_val_df_final = calculate_P_values(lab_abnom_df, total_n_row_name = "...")

In [ ]:
lab_abnom_p_val_df_final

In [ ]:
lab_abnom_p_val_df_final.to_csv("...", index = True)

# Compare Demographics and Comorbidities

In [ ]:
%store -r Comorbidities_dict

In [ ]:
comorbidity_space = list(Comorbidities_dict.keys())

In [ ]:
comorbidity_names = [disease.replace("...", "...").title() for disease in comorbidity_space]
index = ["...", 
         "...", "...", "...",
         "...",  "...", "...",
         "...", "...",
         "...", "...", "...",
         *comorbidity_names]
DEMO_df = pd.DataFrame(0, columns = ["...", *cluster_names[:len(cluster_names)//2], 
                                    "...", *cluster_names[len(cluster_names)//2:]], 
                       index = index)

In [ ]:
# fill in total patient number
for name in cluster_names:
    cluster_idx = cluster_mapping[name]
    DEMO_df.loc["...", name] = len(all_patients[all_patients.CLUSTERS == cluster_idx])

In [ ]:
# fill in age
for name in cluster_names:
    cluster_idx = cluster_mapping[name]
    this_cluster = all_patients[all_patients.CLUSTERS == cluster_idx]
    DEMO_df.loc["...", name] = age_distr["..."][name]
    DEMO_df.loc["...", name] = age_distr["..."][name]
    DEMO_df.loc["...", name] = age_distr["..."][name]

In [ ]:
all_patients["..."] = (all_patients.SEX == "...").astype(int)
#impute BMI jointly with age, sex, and labs
BMI_info_cols = ["...", "...", "...", *filtered_lab_space]
# Use MICE to impute BMI values
imputer = IterativeImputer(missing_values=np.nan, 
                           max_iter=10000, random_state=42)
# should only fit on train data in case for data leakage
all_patients.loc[:, BMI_info_cols] = \
imputer.fit_transform(all_patients.loc[:, BMI_info_cols])
# convert data type
all_patients.loc[:, ["...", *filtered_lab_space]] = \
all_patients.loc[:, ["...", *filtered_lab_space]].astype(np.float64)
all_patients.loc[:, "..."] = all_patients.loc[:, "..."].astype(int)
all_patients.drop("...", axis = 1, inplace = True)

In [ ]:
# fill in BMI
for name in cluster_names:
    cluster_idx = cluster_mapping[name]
    this_cluster = all_patients[all_patients.CLUSTERS == cluster_idx]
    DEMO_df.loc["...", name] = len(this_cluster[this_cluster.BMI < 18.5])
    DEMO_df.loc["...", name] = len(this_cluster[(this_cluster.BMI >= 18.5) & \
                                                          (this_cluster.AGE < 40)])
    DEMO_df.loc["...", name] = len(this_cluster[this_cluster.BMI >= 40])

In [ ]:
# fill in sex
for name in cluster_names:
    cluster_idx = cluster_mapping[name]
    this_cluster = all_patients[all_patients.CLUSTERS == cluster_idx]
    DEMO_df.loc["...", name] = len(this_cluster[this_cluster.SEX == "..."])
    DEMO_df.loc["...", name] = len(this_cluster[this_cluster.SEX >= "..."])

In [ ]:
# fill in race
for name in cluster_names:
    cluster_idx = cluster_mapping[name]
    this_cluster = all_patients[all_patients.CLUSTERS == cluster_idx]
    DEMO_df.loc["...", name] = len(this_cluster[this_cluster.RACE == "..."])
    DEMO_df.loc["...", name] = len(this_cluster[this_cluster.RACE == "..."])
    DEMO_df.loc["...", name] = len(this_cluster[(this_cluster.RACE != "...") & \
                                                  (this_cluster.RACE != "...")])

In [ ]:
# fill in comorbidities and history
for name in cluster_names:
    cluster_idx = cluster_mapping[name]
    this_cluster = all_patients[all_patients.CLUSTERS == cluster_idx]
    for disease in comorbidity_space:
        disease_row_name = disease.replace("...", "...").title()
        DEMO_df.loc[disease_row_name, name] = len(this_cluster[this_cluster[disease] == True])

In [ ]:
DEMO_df.loc[:, "..."] = DEMO_df.loc[:, cluster_names[:len(cluster_names)//2]].sum(axis = 1)
DEMO_df.loc[:, "..."] = DEMO_df.loc[:, cluster_names[len(cluster_names)//2:]].sum(axis = 1)

In [ ]:
DEMO_df_final = DEMO_df.apply(add_percentage, axis = 0)

In [ ]:
DEMO_df_final

In [ ]:
DEMO_df_final.to_csv("...", index = True)

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("...")
    DEMO_p_val_df_final = calculate_P_values(DEMO_df, total_n_row_name = "...")

In [ ]:
DEMO_p_val_df_final

In [ ]:
DEMO_p_val_df_final.to_csv("...", index = True)

# CKD Incidence

Among patients without CKD before, how long it took until they got CKD? focus on 1-year CKD incidence rates

In [ ]:
from A_Label_AKI_Onsets import read_DX
from D_Matching_alt import reverse_comorbidity_dict

In [ ]:
all_patients["..."] = all_patients[["...", 
                                            "...", 
                                            "...", 
                                            "...", 
                                            "..."]].any(axis=1)

# for this one, we can only do in five centers since "...", "...", "..." do not have after-discharge DX
non_CKD_patients = all_patients[(all_patients.CKD_HISTORY == False) & \
                                ~(all_patients.CENTER_NAME.isin(["...", "...", "..."]))].copy(deep = True)
non_CKD_patients.reset_index(drop = True, inplace = True)
non_CKD_patients["..."] = pd.to_datetime(non_CKD_patients["..."])
non_CKD_patients["..."] = pd.to_datetime(non_CKD_patients["..."])

In [ ]:
DX_dict = read_DX(ct_names, raw_path)

In [ ]:
def format_CKD_DX_dict(DX_dict, pat_df, reverse_CKD_dict):
    processed_DX_dict = dict()
    ct_missing_DX_DATE = ["...", "...", "..."]
    
    for ct_name, DX_df in tqdm(DX_dict.items()):
        # filter those not CKD DX
        DX_df = DX_df[DX_df.DX.isin(list(reverse_CKD_dict.keys()))]
        DX_df.loc[:, "..."] = DX_df.loc[:, "..."].astype(str)
        pat_ct_df = pat_df[pat_df.CENTER_NAME == ct_name]
        pat_ct_df = pat_ct_df.merge(DX_df[["...", "...", "...", "...", "..."]], 
                                    on = "...", how = "...")
        pat_ct_df.dropna(subset=["..."], inplace = True)
        
        if ct_name not in ct_missing_DX_DATE:
            pat_ct_df["..."] = pd.to_datetime(pat_ct_df["..."], format = "...")
            pat_ct_df["..."] = pat_ct_df["..."].dt.strftime("...")
            pat_ct_df["..."] = pd.to_datetime(pat_ct_df["..."], format = "...")
        else:
            pat_ct_df.loc[:, "..."] = pat_ct_df.loc[:, "..."] + \
            pd.to_timedelta(pat_ct_df.loc[:, "..."], unit="...")
        
        pat_ct_df = pat_ct_df.loc[(pat_ct_df.DX_DATE > pat_ct_df.DISCHARGE_DATE) & \
                              (pat_ct_df.DX_DATE < pat_ct_df.DISCHARGE_DATE + pd.Timedelta(days=365)), :]
        
        # we only care about incidence date
        pat_ct_df.drop(["...", "..."], axis = 1, inplace = True)
        pat_ct_df = pat_ct_df.loc[:, ["...", "...", "...", "..."]]
        processed_DX_dict[ct_name] = pat_ct_df
    return processed_DX_dict

In [ ]:
CKD_dict = {
    "...": {"...": ["..."], "...":["..."]},
    "...": {"...": ["..."], "...":["..."]},
    "...": {"...": ["..."], "...":["...", "...", "...", "..."]},
    "...": {"...": ["..."], "...":["..."]},
    "...": {"...": ["..."], "...":["..."]},
}
reverse_CKD_dict = reverse_comorbidity_dict(CKD_dict)

In [ ]:
future_CKD_dict = format_CKD_DX_dict(DX_dict, non_CKD_patients, reverse_CKD_dict)

In [ ]:
future_CKD_df = concat_dfs_to_one(future_CKD_dict)

In [ ]:
# we only take the earlister CKD incidence
future_CKD_df = future_CKD_df.sort_values(by=pat_id_cols + ["..."], ascending=True)

In [ ]:
future_CKD_df = future_CKD_df.groupby(pat_id_cols).first().reset_index()
future_CKD_df.rename(columns = {"...": "..."}, inplace = True)

In [ ]:
non_CKD_patients = non_CKD_patients.merge(future_CKD_df, on = pat_id_cols, how = "...")

In [ ]:
non_CKD_patients["..."] = (non_CKD_patients["..."] - non_CKD_patients["..."]).dt.days
non_CKD_patients["..."] = (non_CKD_patients["..."] < 365).astype(int)
non_CKD_patients["..."] = non_CKD_patients["..."].apply(lambda x: 365 if pd.isna(x) or x > 365 else x)

In [ ]:
# we observe a sharp increse in the first 100 days of class 3
class_2_CKD = non_CKD_patients[(non_CKD_patients.CLUSTERS == cluster_mapping["..."]) & \
                      (non_CKD_patients.CKD_INCIDENCE_1_YEAR_EVENT == 1)]
class_2_CKD_first_100 = class_2_CKD[class_2_CKD.CKD_INCIDENCE_1_YEAR_TIME <= 100]

class_3_CKD = non_CKD_patients[(non_CKD_patients.CLUSTERS == cluster_mapping["..."]) & \
                      (non_CKD_patients.CKD_INCIDENCE_1_YEAR_EVENT == 1)]
class_3_CKD_first_100 = class_3_CKD[class_3_CKD.CKD_INCIDENCE_1_YEAR_TIME <= 100]

print(len(class_2_CKD_first_100) / len(class_2_CKD))
print(len(class_3_CKD_first_100) / len(class_3_CKD))

In [ ]:
non_CKD_AKI_patients = non_CKD_patients.loc[non_CKD_patients.CLUSTERS.isin([cluster_mapping["..."], cluster_mapping["..."], cluster_mapping["..."]]), :]

In [ ]:
CKD_adjust_variables_3 = adjust_variables_2 + ["...", "...", "..."]
CKD_adjust_variables_4 = CKD_adjust_variables_3 + ["..."]

In [ ]:
with contextlib.redirect_stdout(open(os.devnull, "...")), \
     contextlib.redirect_stderr(open(os.devnull, "...")), \
     warnings.catch_warnings():
    
    warnings.simplefilter("...")     # Suppress warnings like ConvergenceWarning
    np.seterr(all="...")             # Suppress numpy warnings like overflow
    target_list = ["..."]
    CKD_adjust_var_list = [adjust_variables_1, adjust_variables_2, CKD_adjust_variables_3, CKD_adjust_variables_4]
    outcome_df = populate_AKI_outcome(outcome_df, non_CKD_AKI_patients, target_list, CKD_adjust_var_list)

In [ ]:
with contextlib.redirect_stdout(open(os.devnull, "...")), \
     contextlib.redirect_stderr(open(os.devnull, "...")), \
     warnings.catch_warnings():
    
    warnings.simplefilter("...")     # Suppress warnings like ConvergenceWarning
    np.seterr(all="...")             # Suppress numpy warnings like overflow
    target_list = ["..."]
    CKD_adjust_var_list = [adjust_variables_1, adjust_variables_2, CKD_adjust_variables_3, CKD_adjust_variables_4]
    outcome_df = populate_AKI_vs_non_AKI_outcome(outcome_df, non_CKD_patients, target_list, CKD_adjust_var_list)

In [ ]:
outcome_df

In [ ]:
outcome_df.to_csv("...", index = True)

# Plot Line Charts

1. 30-day mortality + in-hospital mortality     
2. 1-year mortality
3. 1-year dialysis or RRT
4. 1-year CKD incidence

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
def plot_outcomes(ax, pat_df, col_to_plot, time_length, title, plot_in_hos_death = False):
    n_clusters = len(pat_df.CLUSTERS.unique())
    
    if plot_in_hos_death:
        Hospital_death_rates = []
        
    for cluster_idx in range(n_clusters):
        death_rate_series = [0]
        this_cluster = pat_df[pat_df.CLUSTERS == cluster_idx]
        this_cluster_n = this_cluster.shape[0]
        
        # collect in-hospital mortality rates of each cluster
        if plot_in_hos_death:
            this_cluster_in_hos_death = this_cluster[this_cluster[col_to_plot] < 1]
            this_cluster_hos_death_rate = this_cluster_in_hos_death.shape[0] / this_cluster_n
            Hospital_death_rates.append(this_cluster_hos_death_rate)
            print(reversed_cluster_mapping[cluster_idx], "...", this_cluster_hos_death_rate)
        
        # collect culmulative rates of each cluster
        for day in range(1, time_length + 1):
            cum_death_this_day = this_cluster[(this_cluster[col_to_plot] <= day) & (this_cluster[col_to_plot] >= 1)]
            cum_death_rate_this_day = cum_death_this_day.shape[0] / this_cluster_n
            death_rate_series.append(cum_death_rate_this_day)
        
        # plot culmulative rates
        ax.step([day for day in range(time_length + 1)], np.array(death_rate_series) * 100,
                 label = reversed_cluster_mapping[cluster_idx], linestyle = "...", where = "...", 
                 color = new_color_key[cluster_idx])
    
    # add figure legend
    ax.legend(loc = "...")
    ax.set_xlabel("...")
    ax.set_ylabel("...")
    ax.set_title(title, fontsize = 15)
    ax.grid(axis="...", linestyle="...", alpha=0.4)
    
    # plot in-hospital death as small subplot
    if plot_in_hos_death:
        ax_small = ax.inset_axes([0.74, 0.01, 0.25, 0.23])
        for cluster_idx in range(n_clusters):
            ax_small.bar(cluster_idx, 
                         Hospital_death_rates[cluster_idx] * 100, 
                         width = 0.6, color = new_color_key[cluster_idx])
            ax_small.set_ylabel("...", fontsize = 9)
            ax_small.set_title("...", fontsize = 9)
            ax_small.tick_params(axis="...", labelsize=6)
        ax_small.set_xticks([])
        ax_small.grid(axis="...", linestyle="...", alpha=0.3)

In [ ]:
figure, axs = plt.subplots(2, 2, figsize = (12, 10))

plot_outcomes(axs[0, 0], all_patients, "...", 30, 
              "...", plot_in_hos_death = True)

plot_outcomes(axs[0, 1], all_patients, "...", 365, 
              "...", plot_in_hos_death = False)

plot_outcomes(axs[1, 0], all_patients, "...", 365, 
              "...", plot_in_hos_death = False)

plot_outcomes(axs[1, 1], non_CKD_patients, "...", 365, 
              "...", plot_in_hos_death = False)

# Adding sequential labels (A, B, C, D) to each subplot
labels = ["...", "...", "...", "..."]
positions = [(0, 0), (0, 1), (1, 0), (1, 1)]

for label, pos in zip(labels, positions):
    axs[pos].text(-0.1, 1.1, label, transform=axs[pos].transAxes, 
                  fontsize=16, fontweight="...", va="...", ha="...")
    
plt.tight_layout()
plt.savefig("...", format="...", dpi = figure_dpi)
plt.show()

# Supplimentary Figures

1. Subphenotypes distribution among centers.   
2. Laboraroty test results violin plots.  

In [ ]:
import seaborn as sns

In [ ]:
# Create the ordered category based on the masked_site_names dictionary
masked_site_names = {
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
}

# Group by "..." and "..." to get the counts
sb_counts_ct = all_patients.groupby(["...", "..."]).size().reset_index(name="...")

# Map the original "..." to the masked site names
sb_counts_ct["..."] = sb_counts_ct["..."].map(masked_site_names)

# Define the desired order for the X-axis (Site A, Site B, etc.)
order = ["...", "...", "...", "...", "...", "...", "...", "..."]

# Convert "..." to a categorical type with the specified order
sb_counts_ct["..."] = pd.Categorical(sb_counts_ct["..."], categories=order, ordered=True)

# Plot the barplot, now it will be ordered by "..."
plt.figure(figsize=(9,6))
barplot = sns.barplot(x="...", y="...", hue="...", data=sb_counts_ct, palette=new_color_key)

# Replace legend labels
handles, labels = barplot.get_legend_handles_labels()
barplot.legend(handles, ["...", "...", "...", "...", "...", "..."], title="...")

# Set axis labels and title
barplot.set_xlabel("...", fontsize=13)
barplot.set_ylabel("...", fontsize=13)
plt.title("...", fontsize=15)

# Add grid and layout settings
plt.grid(axis="...", linestyle="...", alpha=0.2)
plt.tight_layout()

# Save the figure
plt.savefig("...", format="...", dpi=figure_dpi)

# Show the plot
plt.show()

In [ ]:
lab_unit_dict = \
{
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...", 
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    "...": "...",
    
}

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(18, 16))  # Create a 3x4 grid of subplots
axes = axes.flatten()  # Flatten the 2D array of axes for easy iteration
all_patients.loc[:, "..."] = all_patients.loc[:, "..."]
cols_to_plot = ["..."] + filtered_lab_space


for i, test in enumerate(cols_to_plot):

    if i >= 16:  # Skip if more than 12 plots are required
        break

    percentile_975 = all_patients[test].quantile(0.975)
    percentile_025 = all_patients[test].quantile(0.025)

    filtered_lab_df = all_patients[(all_patients[test] <= percentile_975) & \
                                   (all_patients[test] >= percentile_025)]

    sns.violinplot(ax=axes[i], x="...", y=test, data=filtered_lab_df, 
                   showmedians=True, showextrema=False, palette=new_color_key)

    axes[i].set_xlabel("...")
    axes[i].set_ylabel(lab_unit_dict[test], fontsize = 12)
    axes[i].set_title(test, fontsize = 17)
    axes[i].set_xticks(list(reversed_cluster_mapping.keys()))
    axes[i].set_xticklabels(list(reversed_cluster_mapping.values()), fontsize = 10)


plt.tight_layout()
plt.savefig("...", format="...", dpi = figure_dpi)
plt.show()